In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pdb
import import_ipynb
from main import TensorflowCNN

importing Jupyter notebook from main.ipynb


In [2]:
IMAGE_SIZE = [20, 20]
IPT_SIZE = 30
MODEL_PATH = 'model'
PRECISION_ERROR = 1e-5
MIN_ACCURACY = 0.5
BATCH_SIZE = 100

INFERENCE_SCORE = 0.4
UPDATE_SCORE = 0.3
RESTORE_SCORE = 0.3

In [3]:
def check(seed, channels, kernel_sizes, learning_rate):
    # parameter validation check
    if len(channels) != len(kernel_sizes) + 1 or \
        len(np.shape(kernel_sizes)) != 2 or np.shape(kernel_sizes)[-1] != 2 or \
        len(np.shape(channels)) != 1:
        raise ValueError
    if np.array(channels).dtype != np.int or np.array(kernel_sizes).dtype != np.int:
        raise ValueError
    if type(seed) != int or type(learning_rate) != float:
        raise ValueError
        
    # build reference model
    tf.random.set_seed(seed)
    reference_model = tf.keras.models.Sequential()
    for layer_idx, size in enumerate(kernel_sizes):
        in_channels, out_channels = channels[layer_idx], channels[layer_idx + 1]
        height, width = size
        layer = tf.keras.layers.Conv2D(
            filters=out_channels, 
            kernel_size=[height, width], 
            use_bias = False, 
            input_shape=[None, None, in_channels], 
            activation='relu', 
            padding='valid')
        reference_model.add(layer)
    reference_model.add(tf.keras.layers.GlobalAvgPool2D())
    reference_model.add(tf.keras.layers.Softmax())
    
    # compile reference model
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    reference_model.compile(optimizer=optimizer, loss=loss_fn)
    
    # target model
    filter_list = np.concatenate([layer.weights for layer in reference_model.layers])
    filter_list = [tf.Variable(variable) for variable in filter_list]
    target_model = TensorflowCNN(filter_list, learning_rate)
    
    # check
    np.random.seed(seed)
    images = np.random.rand(IPT_SIZE, *IMAGE_SIZE, channels[0])
    labels = np.random.randint(0, channels[-1], IPT_SIZE)
    score = 0
    
    # check inference
    error = tf.reduce_sum(tf.abs(reference_model(images) - target_model(images)))
    if error <= PRECISION_ERROR:
        score += INFERENCE_SCORE
        
    # check training
    target_model.save()
    reference_model.save('reference_model')
    target_model.fit(images, labels, 1)
    reference_model.fit(images, labels, IPT_SIZE, 1)
    error = tf.reduce_sum(tf.abs(reference_model(images) - target_model(images)))
    if error <= PRECISION_ERROR:
        score += UPDATE_SCORE
        
    # check restore
    target_model.restore()
    reference_model = tf.keras.models.load_model('reference_model')
    error = tf.reduce_sum(tf.abs(reference_model(images) - target_model(images)))
    if error <= PRECISION_ERROR:
        score += RESTORE_SCORE
    return score

def evaluate(testset):
    # load model
    loaded_model = None
    try:
        loaded_model = tf.saved_model.load(MODEL_PATH)
    except:
        return 0.0
    
    # evaluate
    metric = tf.keras.metrics.SparseCategoricalAccuracy()
    for batch in testset.batch(BATCH_SIZE):
        image, label = batch['image'], batch['label']
        image = tf.cast(image, tf.float32)
        pred = loaded_model(image)
        metric.update_state(label, pred)
    if metric.result().numpy() < MIN_ACCURACY:
        return 0.0
    
    return 1.0

In [4]:
score = 0
score += check(1, [4, 5], [[3,3]], 1e-3) * 5.0

testset = tfds.load('cifar100', split='test')
score += evaluate(testset) * 5.0

ValueError: setting an array element with a sequence.